In [201]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain
import os
from dotenv import load_dotenv, find_dotenv


## Charger les documents

In [202]:
long_loader = UnstructuredFileLoader("docs-to-load/arrete-07-avril.txt")
long_doc = long_loader.load() 

### Obtenir le nombre de mots dans chaque document

In [203]:
def doc_preview(docs):
    print (f'Tu as {len(docs)} document(s)')

    #we split the doc word by word with using a space separator ```doc.page_content.split(' ')````
    # we get the number of words with ```len()````
    # then we loop over the docs to get the number of words separated by a space
    num_words = sum([len(doc.page_content.split(' ')) for doc in docs])

    #we print the number of words in the doc
    print (f'Tu as en gros {num_words} mots dans ton document')
    
    print()
    
    #we preview the first document 
    print (f'Aperçu: {docs[0].page_content.split(". ")}[0]')

In [204]:
doc_preview(long_doc)

Tu as 1 document(s)
Tu as en gros 3261 mots dans ton document

Aperçu: ["Arrêté du 7 avril 2020 relatif aux modalités d'admission aux formations conduisant aux diplômes d'Etat d'aide-soignant et d'auxiliaire de puériculture Dernière mise à jour des données de ce texte : 12 juin 2023\n\nTITRE 1ER : DISPOSITIONS GÉNÉRALES (Articles 1 à 8 ter) Article 1 Modifié par Arrêté du 12 avril 2021 - art", "1\n\nI.- Les formations conduisant au diplôme d'Etat d'aide-soignant et au diplôme d'Etat d'auxiliaire de puériculture sont accessibles, sans condition de diplôme, par les voies suivantes :\n\n1° La formation initiale, dans les conditions fixées par le présent arrêté ;\n\n2° La formation professionnelle continue, dans les conditions fixées par le présent arrêté ;\n\n3° La validation, partielle ou totale, des acquis de l'expérience, dans les conditions fixées par arrêté du ministre chargé de la santé.\n\nLes candidats doivent être âgés de dix-sept ans au moins à la date d'entrée en formation.\n\n

## Charger le modèle de langage

In [205]:
from langchain_openai.llms import OpenAI

In [206]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [207]:
from langchain_openai import ChatOpenAI

## Découper le long texte en morceaux

In [208]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [209]:
textsplitter = RecursiveCharacterTextSplitter (chunk_size=400, chunk_overlap=0)

In [210]:
document_long_morceaux = textsplitter.split_documents(long_doc)

In [211]:
doc_preview(document_long_morceaux)
print(f" \n ⭐️ Remarque : au lieu d'avoir {len(long_doc)} document, nous avons maintenant {len(document_long_morceaux)} plus petits documents")

Tu as 76 document(s)
Tu as en gros 3326 mots dans ton document

Aperçu: ["Arrêté du 7 avril 2020 relatif aux modalités d'admission aux formations conduisant aux diplômes d'Etat d'aide-soignant et d'auxiliaire de puériculture Dernière mise à jour des données de ce texte : 12 juin 2023\n\nTITRE 1ER : DISPOSITIONS GÉNÉRALES (Articles 1 à 8 ter) Article 1 Modifié par Arrêté du 12 avril 2021 - art", '1'][0]
 
 ⭐️ Remarque : au lieu d'avoir 1 document, nous avons maintenant 76 plus petits documents


## Résumer puis utiliser les liens et la réduction pour éviter les problèmes de dépassements de tokens dans les textes longs

### Résumé

In [212]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain


In [213]:
llm = ChatOpenAI(temperature=0)

## ❌ 👎🏽 Essai de résumé avec "load_summarize_chain" (donne le résumé en anglais!!)

- tutoriel YT de Greg Kamradt : https://www.youtube.com/watch?v=f9_BWhCI4ZoGithub
- Github: https://github.com/gkamradt/langchain-tutorials/blob/main/chains/Chain%20Types.ipynb

In [221]:
# Define prompt
prompt_template = """Ecris un résumé concis de ce qui suit :
"{long_doc_chunked}"
RESUME CONCIS:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

chain = load_summarize_chain(llm, chain_type="map_reduce")

to_translate = chain.invoke(document_long_morceaux[:3])

to_translate


{'input_documents': [Document(page_content="Arrêté du 7 avril 2020 relatif aux modalités d'admission aux formations conduisant aux diplômes d'Etat d'aide-soignant et d'auxiliaire de puériculture Dernière mise à jour des données de ce texte : 12 juin 2023\n\nTITRE 1ER : DISPOSITIONS GÉNÉRALES (Articles 1 à 8 ter) Article 1 Modifié par Arrêté du 12 avril 2021 - art. 1", metadata={'source': 'docs-to-load/arrete-07-avril.txt'}),
  Document(page_content="I.- Les formations conduisant au diplôme d'Etat d'aide-soignant et au diplôme d'Etat d'auxiliaire de puériculture sont accessibles, sans condition de diplôme, par les voies suivantes :\n\n1° La formation initiale, dans les conditions fixées par le présent arrêté ;\n\n2° La formation professionnelle continue, dans les conditions fixées par le présent arrêté ;", metadata={'source': 'docs-to-load/arrete-07-avril.txt'}),
  Document(page_content="3° La validation, partielle ou totale, des acquis de l'expérience, dans les conditions fixées par ar

In [223]:
from deep_translator import GoogleTranslator
translated = GoogleTranslator(source='auto',target='fr').translate(str(to_translate))
translated

'{\'input_documents\': [Document(page_content="Arrêté du 7 avril 2020 relatif aux modalités d\'admission aux formations conduisant aux diplômes d\'Etat d\'aide-soignant et d\'auxiliaire de puériculture Dernière mise à jour des données de ce texte : 12 juin 2023\\n\\nTITRE 1ER : DISPOSITIONS GÉNÉRALES (Articles 1 à 8 ter) Article 1 Modifié par Arrêté du 12 avril 2021 - art. 1", metadata={\'source\': \'docs-to-load/arrete-07-avril.txt\'}), Document(page_content="I.- Les formations conduisant au diplôme d\'Etat d\'aide-soignant et au diplôme d\'Etat d\'auxiliaire de puériculture sont accessibles, sans condition de diplôme, par les voies suivantes :\\n\\n1° La formation initiale, dans les conditions fixées par le présent arrêté ;\\n\\n2° La formation professionnelle continue, dans les conditions fixées par le présent arrêté ;", metadata={\'source\': \'docs-to-load/arrete-07-avril.txt\'}), Document(page_content="3° La validation, partielle ou totale, des acquis de l\'expérience, dans les co

## ✅ 👍🏽🎉 Essai en détaillant toute la Pipeline équivalente à l'objet "load_summarize_chain" : donne le résumé en français 

source : https://python.langchain.com/v0.1/docs/use_cases/summarization/

In [215]:
# Map
map_template = """Ce qui suit est un ensemble de documents
{document_long_morceaux}
Sur la base de cette liste de long_doc_chunked, identifie les principaux thèmes 
Réponses Utiles:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [216]:
# Reduce
reduce_template = """Ce qui suit est un ensemble de résumés :
{document_long_morceaux}
Prends-les et distille-les dans un résumé final et consolidé des principaux thèmes. 
Réponses Utiles:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [217]:
#Run the Reduce chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

In [218]:
# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="document_long_morceaux"
)

In [219]:
# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4096,
)

In [220]:
#we run the summarization on the first 5 chunks ("docs") of the long doc not to waste the api query usage
reduce_documents_chain.invoke(document_long_morceaux[:3])

{'input_documents': [Document(page_content="Arrêté du 7 avril 2020 relatif aux modalités d'admission aux formations conduisant aux diplômes d'Etat d'aide-soignant et d'auxiliaire de puériculture Dernière mise à jour des données de ce texte : 12 juin 2023\n\nTITRE 1ER : DISPOSITIONS GÉNÉRALES (Articles 1 à 8 ter) Article 1 Modifié par Arrêté du 12 avril 2021 - art. 1", metadata={'source': 'docs-to-load/arrete-07-avril.txt'}),
  Document(page_content="I.- Les formations conduisant au diplôme d'Etat d'aide-soignant et au diplôme d'Etat d'auxiliaire de puériculture sont accessibles, sans condition de diplôme, par les voies suivantes :\n\n1° La formation initiale, dans les conditions fixées par le présent arrêté ;\n\n2° La formation professionnelle continue, dans les conditions fixées par le présent arrêté ;", metadata={'source': 'docs-to-load/arrete-07-avril.txt'}),
  Document(page_content="3° La validation, partielle ou totale, des acquis de l'expérience, dans les conditions fixées par ar